<a href="https://colab.research.google.com/github/MASAYA-KINOSHITA/first-Repository/blob/main/%E6%B0%91%E6%B3%8A%E3%82%B5%E3%83%BC%E3%83%93%E3%82%B9%E3%81%AE%E5%AE%BF%E6%B3%8A%E6%96%99%E9%87%91%E4%BA%88%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#from モジュール名 import クラス名（もしくは関数名や変数名）
import pandas as pd
from pandas import DataFrame
import numpy as np

In [19]:
#データの読み込み
train_df = pd.read_csv("/content/drive/MyDrive/民泊サービスの宿泊料金予測/train/train_data.csv")
test_df = pd.read_csv("/content/drive/MyDrive/民泊サービスの宿泊料金予測/test/test_data.csv")
submit_df = pd.read_csv("/content/drive/MyDrive/民泊サービスの宿泊料金予測/提出用ファイル/submission.csv")
station_list_df = pd.read_csv("/content/drive/MyDrive/民泊サービスの宿泊料金予測/train/station_list.csv")
print(train_df.shape)
print(test_df.shape)

(9990, 13)
(4996, 12)


In [20]:
#データの確認
train_df.head(10)

,id,name,host_id,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365,y
0,1,KiyosumiShirakawa 3min|★SkyTree★|WIFI|Max4|Tre...,242899459,Koto Ku,35.68185,139.80310,Entire home/apt,1,55,2020-04-25,2.21,173,12008
1,2,Downtown Tokyo Iriya next to Ueno,308879948,Taito Ku,35.72063,139.78536,Entire home/apt,6,72,2020-03-25,2.11,9,6667
2,3,"Japan Style,Private,Affordable,4min to Sta.",300877823,Katsushika Ku,35.74723,139.82349,Entire home/apt,1,18,2020-03-23,3.46,288,9923
3,4,4 min to Shinjuku Sta. by train / 2 ppl / Wi-fi,236935461,Shibuya Ku,35.68456,139.68077,Entire home/apt,1,2,2020-04-02,1.76,87,8109
4,5,LICENSED SHINJUKU HOUSE: Heart of the action!,243408889,Shinjuku Ku,35.69840,139.70467,Entire home/apt,1,86,2020-01-30,2.00,156,100390
5,6,"Quiet residential Shinjuku area,8 min on foot Sta",247956261,Shinjuku Ku,35.71273,139.69687,Entire home/apt,2,30,2020-03-29,5.20,335,8404
6,7,5mins walk to sta. /FamilyMat /double room liuqm1,314405566,Arakawa Ku,35.73180,139.79445,Entire home/apt,1,8,2020-04-15,1.69,111,5811
7,8,花川戸(竹)池袋から600米高端民宿(空港送迎ｻｰﾋﾞｽあり),256531617,Toshima Ku,35.73665,139.70821,Entire home/apt,1,7,2020-02-02,0.93,134,19938
8,9,"东京精品民宿5,近新宿､渋谷､池袋､銀座､秋葉原､上野",114995297,Toshima Ku,35.73698,139.73243,Private room,1,12,2020-04-27,0.88,89,10216
9,10,【2/F】Asakusabashi Building | for 6-9 ppl,260891041,Taito Ku,35.70020,139.78538,Entire home/apt,2,74,2020-01-26,1.48,0,24826


In [21]:
"""
各カラムの処理をメモする。
・id, host_idは削除する。
・緯度経度は主要駅からの距離の特徴量とする。
・name,last_reviewは一旦削除する。
・reviews_per_monthの欠損については処理せずに進める。
・neighbourhood,room_typeはラベルエンコーディングを行う。
"""

'\n各カラムの処理をメモする。\n・id, host_idは削除する。\n・緯度経度は主要駅からの距離の特徴量とする。\n・name,last_reviewは一旦削除する。\n・reviews_per_monthの欠損については処理せずに進める。\n・neighbourhood,room_typeはラベルエンコーディングを行う。\n'

In [64]:
#目的変数のみのリストを作成
train_y=train_df["y"]

#目的変数、カテゴリ変数、緯度、経度を除いたリストを作成
train_x=train_df.drop(["minimum_nights","availability_365","id","host_id","y","neighbourhood","room_type","latitude","longitude","name","last_review"],axis=1)
test_x=test_df.drop(["minimum_nights","availability_365","id","host_id","neighbourhood","room_type","latitude","longitude","name","last_review"],axis=1)

In [65]:
# 各カラムの種類の数を確認
for colname in train_x.columns:
  lencol = len(train_x[colname].unique())
  print(colname,lencol)
  

number_of_reviews 261
reviews_per_month 595


In [66]:
#availability_に366以上の値が入っていないかチェックする。
reqd_Index = train_x[train_x['availability_365']>=366].index.tolist()
print(reqd_Index)

KeyError: ignored

In [67]:
#minimum_nightsに366以上の値が入っていないかチェックする。
reqd_Index = train_x[train_x['minimum_nights']>=366].index.tolist()
print(reqd_Index)

KeyError: ignored

In [68]:
# ラベルエンコーディング（OrdinalEncoder）
from sklearn import preprocessing
from sklearn.preprocessing import OrdinalEncoder


oe = preprocessing.OrdinalEncoder()
train_encoded = oe.fit_transform(train_df[["neighbourhood","room_type"]].values)
train_decoded = oe.inverse_transform(train_encoded)
test_encoded = oe.fit_transform(test_df[["neighbourhood","room_type"]].values)
test_decoded = oe.inverse_transform(test_encoded)

print('エンコード結果: ', [train_encoded,test_encoded])
print('元に戻す: ', [train_decoded,test_decoded])

エンコード結果:  [array([[ 9.,  0.],
       [21.,  0.],
       [ 7.,  0.],
       ...,
       [21.,  2.],
       [ 7.,  3.],
       [20.,  1.]]), array([[11.,  2.],
       [22.,  0.],
       [15.,  0.],
       ...,
       [18.,  0.],
       [18.,  3.],
       [18.,  2.]])]
元に戻す:  [array([['Koto Ku', 'Entire home/apt'],
       ['Taito Ku', 'Entire home/apt'],
       ['Katsushika Ku', 'Entire home/apt'],
       ...,
       ['Taito Ku', 'Private room'],
       ['Katsushika Ku', 'Shared room'],
       ['Sumida Ku', 'Hotel room']], dtype=object), array([['Minato Ku', 'Private room'],
       ['Toshima Ku', 'Entire home/apt'],
       ['Setagaya Ku', 'Entire home/apt'],
       ...,
       ['Shinjuku Ku', 'Entire home/apt'],
       ['Shinjuku Ku', 'Shared room'],
       ['Shinjuku Ku', 'Private room']], dtype=object)]


In [69]:
#エンコードしたカテゴリデータにラベル名をつける
train_encoded_df = pd.DataFrame(train_encoded)
train_encoded_df.columns=["neighbourhood","room_type"]
test_encoded_df = pd.DataFrame(test_encoded)
test_encoded_df.columns=["neighbourhood","room_type"]

#エンコードしたカテゴリデータとカテゴリを除いた訓練データを結合
train_x=pd.concat([train_x, train_encoded_df],axis=1)
test_x=pd.concat([test_x, test_encoded_df],axis=1)

In [70]:
#train_dfから緯度経度を取得
train_geo=train_df.loc[:,['latitude','longitude']]
test_geo=test_df.loc[:,['latitude','longitude']]
train_geo.head()

,latitude,longitude
0,35.68185,139.80310
1,35.72063,139.78536
2,35.74723,139.82349
3,35.68456,139.68077
4,35.69840,139.70467


In [71]:
#station_listから東京駅、渋谷駅、新宿駅の行を取得する。
print (station_list_df.loc[(station_list_df['station_name'] == "東京")|(station_list_df['station_name'] == "新宿")|(station_list_df['station_name'] == "渋谷")])

    station_name   longitude   latitude
89            新宿  139.698812  35.688690
90            新宿  139.699187  35.690163
91            新宿  139.699574  35.691435
92            新宿  139.700464  35.689729
93            新宿  139.700711  35.692350
94            渋谷  139.700872  35.658700
95            渋谷  139.701000  35.659066
96            渋谷  139.701238  35.658871
97            渋谷  139.702148  35.658020
98            渋谷  139.702417  35.659545
160           東京  139.764708  35.681753
162           東京  139.766103  35.681391


In [72]:
#複数存在するのでそれぞれの平均を取る。
tokyo_df=station_list_df.loc[(station_list_df["station_name"]=="東京")]
tokyo_df=tokyo_df.mean()
shibuya_df=station_list_df.loc[(station_list_df["station_name"]=="渋谷")]
shibuya_df=shibuya_df.mean()
shinjuku_df=station_list_df.loc[(station_list_df["station_name"]=="新宿")]
shinjuku_df=shinjuku_df.mean()
print(tokyo_df)
print(shibuya_df)
print(shinjuku_df)

longitude    139.765405
latitude      35.681572
dtype: float64
longitude    139.701535
latitude      35.658840
dtype: float64
longitude    139.699750
latitude      35.690473
dtype: float64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


In [73]:
#主要駅の緯度経度のDataFrameを作成
list2 = [["distance_from_Tokyo",  35.681572,  139.765405], ["distance_from_Shibuya", 35.658840,  139.701535],["distance_from_Shinjuku", 35.690473, 139.699750]]
columns = ['location', 'latitude', 'longitude']
locations = pd.DataFrame(data=list2,columns=columns)
locations.head()

,location,latitude,longitude
0,distance_from_Tokyo,35.681572,139.765405
1,distance_from_Shibuya,35.658840,139.701535
2,distance_from_Shinjuku,35.690473,139.699750


In [74]:
#geopyを用いて各駅からの距離を算出
from geopy.distance import geodesic

for i in range(len(locations)) :
  train_geo[locations['location'][i]] = train_geo.apply(lambda x:geodesic((x["latitude"], x["longitude"]), (locations['latitude'][i], locations['longitude'][i])).km, axis=1)

for i in range(len(locations)) :
  test_geo[locations['location'][i]] = test_geo.apply(lambda x:geodesic((x["latitude"], x["longitude"]), (locations['latitude'][i], locations['longitude'][i])).km, axis=1)

train_geo.head()

,latitude,longitude,distance_from_Tokyo,distance_from_Shibuya,distance_from_Shinjuku
0,35.68185,139.80310,3.412464,9.543268,9.403993
1,35.72063,139.78536,4.694873,10.226027,8.439178
2,35.74723,139.82349,8.983122,14.764682,12.845811
3,35.68456,139.68077,7.668589,3.417305,1.839037
4,35.69840,139.70467,5.805862,4.398468,0.985834


In [75]:
#train_geoとtrain_xを結合
train_x=pd.concat([train_x, train_geo],axis=1)
test_x=pd.concat([test_x, test_geo],axis=1)
train_x.head()

,number_of_reviews,reviews_per_month,neighbourhood,room_type,latitude,longitude,distance_from_Tokyo,distance_from_Shibuya,distance_from_Shinjuku
0,55,2.21,9.0,0.0,35.68185,139.80310,3.412464,9.543268,9.403993
1,72,2.11,21.0,0.0,35.72063,139.78536,4.694873,10.226027,8.439178
2,18,3.46,7.0,0.0,35.74723,139.82349,8.983122,14.764682,12.845811
3,2,1.76,16.0,0.0,35.68456,139.68077,7.668589,3.417305,1.839037
4,86,2.00,18.0,0.0,35.69840,139.70467,5.805862,4.398468,0.985834


In [76]:
test_x.head()

,number_of_reviews,reviews_per_month,neighbourhood,room_type,latitude,longitude,distance_from_Tokyo,distance_from_Shibuya,distance_from_Shinjuku
0,0,NaN,11.0,2.0,35.67131,139.73285,3.159514,3.154932,3.674110
1,21,1.94,22.0,0.0,35.73014,139.71739,6.922434,8.040077,4.681738
2,14,0.82,15.0,0.0,35.66193,139.66540,9.312631,3.289890,4.438457
3,16,1.19,22.0,0.0,35.73603,139.74794,6.245594,9.538883,6.675743
4,2,0.19,21.0,0.0,35.72126,139.78320,4.688782,10.129455,8.288622


In [77]:
# 相関係数
import scipy.stats as st

corrs = []
for c in train_x.columns:
    corr = np.corrcoef(train_x[c], train_y)[0, 1]
    corrs.append(corr)
corrs = np.array(corrs)

# 重要度の上位を出力する（上位11まで）

idx = np.argsort(np.abs(corrs))[::-1]
top_cols, top_importances = train_x.columns.values[idx][:11], corrs[idx][:11]
print(top_cols, top_importances)


['reviews_per_month' 'room_type' 'distance_from_Tokyo' 'longitude'
 'neighbourhood' 'number_of_reviews' 'distance_from_Shinjuku'
 'distance_from_Shibuya' 'latitude'] [        nan -0.11858969 -0.07206847  0.06792723 -0.05031426 -0.04129568
  0.02614471  0.02219679  0.01442708]


In [78]:
# 学習データを学習データとバリデーションデータに分ける
from sklearn.model_selection import KFold

kf = KFold(n_splits=4, shuffle=True, random_state=71)
tr_idx, va_idx = list(kf.split(train_x))[0]
tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

In [79]:
# xgboostの実装
# -----------------------------------
import xgboost as xgb
from sklearn.metrics import log_loss

# 特徴量と目的変数をxgboostのデータ構造に変換する
dtrain = xgb.DMatrix(train_x, label=train_y)
dvalid = xgb.DMatrix(va_x,label=va_y)
dtest = xgb.DMatrix(test_x)

In [80]:
# 学習データとバリデーションデータのスコアのモニタリング
# -----------------------------------
# モニタリングをloglossで行い、アーリーストッピングの観察するroundを20とする
params = {
    #'booster': 'gbtree',
    'objective': 'reg:squarederror',
    #'eta': 0.1,
    #'gamma': 0.0,
    #'alpha': 0.0,
    #'lambda': 1.0,
    #'min_child_weight': 1,
    #'max_depth': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.8,
    'silent':1,
    'random_state': 71,
    #上記で取得したパラメータ
}
num_round = 5000
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
model = xgb.train(params, dtrain, num_round, evals=watchlist,
                  early_stopping_rounds=20)


[0]	train-rmse:62971.1	eval-rmse:51815.9
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 20 rounds.
[1]	train-rmse:57647	eval-rmse:49157.9
[2]	train-rmse:54697.8	eval-rmse:47523.5
[3]	train-rmse:52611.1	eval-rmse:46468.1
[4]	train-rmse:51159.2	eval-rmse:45961.8
[5]	train-rmse:49948.5	eval-rmse:45526.7
[6]	train-rmse:48377.2	eval-rmse:44233.8
[7]	train-rmse:47644.3	eval-rmse:43678.2
[8]	train-rmse:46909.5	eval-rmse:43293.7
[9]	train-rmse:46018.7	eval-rmse:41219.9
[10]	train-rmse:45390.3	eval-rmse:40934.6
[11]	train-rmse:44959.9	eval-rmse:40790.9
[12]	train-rmse:43614.1	eval-rmse:38969.3
[13]	train-rmse:42118	eval-rmse:38140.7
[14]	train-rmse:41826.7	eval-rmse:38072.7
[15]	train-rmse:41364.9	eval-rmse:37916.8
[16]	train-rmse:40385.9	eval-rmse:37450.1
[17]	train-rmse:40117.1	eval-rmse:37182.2
[18]	train-rmse:39213.7	eval-rmse:36907.6
[19]	train-rmse:38173	eval-rmse:35898.5
[20]	train-rmse:36839.3	eval-rmse

In [81]:
# 構築したモデルの保存
import pickle
file = '/content/drive/MyDrive/民泊サービスの宿泊料金予測/モデル/xgboost_notuning1.pkl'
pickle.dump(model, open(file, 'wb'))

# 学習済みモデルを削除
del model

In [82]:
# 構築したモデルの読み込み
import pickle
filename = '/content/drive/MyDrive/民泊サービスの宿泊料金予測/モデル/xgboost_notuning1.pkl'
loaded_model = pickle.load(open(filename, 'rb'))

In [83]:
id_df=submit_df.iloc[:,0]
id_df.head()

0    1
1    2
2    3
3    4
4    5
Name: id, dtype: int64

In [84]:
pred=loaded_model.predict(dtest,ntree_limit=loaded_model.best_ntree_limit)

In [85]:
pred[pred<0]=0

In [88]:
pred_df = pd.DataFrame(pred)
pred_df.head()

,0
0,31461.205078
1,15984.518555
2,6336.778809
3,316568.093750
4,12363.938477


In [89]:
output_df=pd.concat([id_df, pred_df],axis=1)
output_df.columns=['id','y']
output_df.head(5)

,id,y
0,1,31461.205078
1,2,15984.518555
2,3,6336.778809
3,4,316568.093750
4,5,12363.938477


In [91]:
output_df.to_csv('/content/drive/MyDrive/民泊サービスの宿泊料金予測/提出用ファイル/xgboost_notuning1.csv',index=False)